In [305]:
import plotly
import plotly
#import plotly.graph_objs as go
import plotly.figure_factory as ff
import os

import pandas as pd


#correlation analysis

In [239]:
#load SMA's data
sse = pd.read_csv('sample_data_SMA/sma_sentiment_equities.csv')

In [241]:
sse['Date'] = pd.to_datetime(sse['center-date'], format = '%m/%d/%y')
sse.rename(columns = {'ticker' : 'Symbol'}, inplace = True)
sse = sse.iloc[ : , [18, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]

In [243]:
sma_symbols = list(set(list(sse['Symbol'])))

In [333]:
final_res_corr = pd.DataFrame([])
for i, item in enumerate(sma_symbols):
    temp = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item))
    temp.insert(loc = 4, column = 'Close Price Change_Forward_3day', value = temp['Close Price'].diff(3))
    temp.insert(loc = 5, column = 'Close Price Percent Change_Forward_3day(%)', \
          value = (temp['Close Price'].pct_change(3)*100).round(2))
    temp['Close Price Change_Forward_3day'] = temp['Close Price Change_Forward_3day'].shift(-3)
    temp['Close Price Percent Change_Forward_3day(%)'] = temp['Close Price Percent Change_Forward_3day(%)'].shift(-3)
    temp.insert(loc = 4, column = 'Close Price Change_Forward_1day', value = temp['Close Price'].diff(1))
    temp.insert(loc = 5, column = 'Close Price Percent Change_Forward_1day(%)', \
          value = (temp['Close Price'].pct_change(1)*100).round(2))
    temp['Close Price Change_Forward_1day'] = temp['Close Price Change_Forward_1day'].shift(-1)
    temp['Close Price Percent Change_Forward_1day(%)'] = temp['Close Price Percent Change_Forward_1day(%)'].shift(-1) 

    temp.columns = ['Date', 'Symbol', 'PageView', 'ClosePrice', 'price_change1', 'price_percent_change1(%)', \
                    'price_change_3', 'price_percent_change3(%)', 'pv_change_in_std1', 'pv_change_in_std3']
    temp1 = sse.loc[sse['Symbol'] == item].copy()
    temp1['Date'] = temp1['Date'].apply(str).str[:10]
    #merge pageview data with SMA's sentiment data
    final_res = pd.merge(temp, temp1, on = ['Date', 'Symbol'])
    final_res = final_res.dropna()
    if not os.path.exists(os.path.dirname('corr_analysis/{}/'.format(item))):
                    os.makedirs(os.path.dirname('corr_analysis/{}/'.format(item)))
    #plot here
    try:
        fig = ff.create_scatterplotmatrix(final_res.loc[:, ['price_percent_change1(%)', 'price_percent_change3(%)',\
                                                            's', 's-mean']], size = 3, \
                                          diag = 'histogram', height = 900, width = 900)
        plotly.offline.plot(fig, filename = 'corr_analysis/{}/scatterplot matrix for price_change(%) vs sentiment.html'.\
                            format(item), auto_open = False)

        fig1 = ff.create_scatterplotmatrix(final_res.loc[:, ['pv_change_in_std1', 'pv_change_in_std3', \
                                                              'sv-score']], size = 3, \
                                          diag = 'histogram', height = 600, width = 600)
        plotly.offline.plot(fig1, \
                            filename = 'corr_analysis/{}/scatterplot matrix for pageview_change_in_std vs sv-score.html'.\
                           format(item), auto_open = False)

        fig2 = ff.create_scatterplotmatrix(final_res.loc[:, ['PageView', 's-volume']])
        plotly.offline.plot(fig2, filename = 'corr_analysis/{}/scatterploe for pageview vs twitter_volume.html'.\
                            format(item), auto_open = False)
    except:
        print(item)
        continue
    corr_table = final_res.corr()
    final_res_corr.loc[i, 'symbol'] = item
    final_res_corr.loc[i, 'corr_pageview_s-volume'] = corr_table.loc['PageView', 's-volume']
    final_res_corr.loc[i, 'corr_pv_change_in_std1_sv-score'] = corr_table.loc['pv_change_in_std1', 'sv-score']
    final_res_corr.loc[i, 'corr_pv_change_in_std3_sv-score'] = corr_table.loc['pv_change_in_std3', 'sv-score']
    final_res_corr.loc[i, 'corr_price_change1(%)_s'] = corr_table.loc['price_percent_change1(%)', 's']
    final_res_corr.loc[i, 'corr_price_change3(%)_s'] = corr_table.loc['price_percent_change3(%)', 's']
    final_res_corr.loc[i, 'corr_price_change1(%)_s-mean'] = corr_table.loc['price_percent_change1(%)', 's-mean']
    final_res_corr.loc[i, 'corr_price_change3(%)_s-mean'] = corr_table.loc['price_percent_change3(%)', 's-mean']
    
#save table   
final_res_corr.to_csv('corr_analysis/correlation_table.csv', index = False)



FOXA
